# ビジネスケース　Audiobooks

### データの読み込み

In [ ]:
import numpy as np

# 前処理にはsklearnライブラリを使っていきます
from sklearn import preprocessing

# データを読み込みます
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# 最初のIDの列と、ターゲットである最後の列を取り除いた上で変数に代入します
unscaled_inputs_all = raw_csv_data[:,1:-1]

# 最後の列のデータをターゲットとして変数に代入していきます
targets_all = raw_csv_data[:,-1]

### データセットのバランシング

In [ ]:
# ターゲットの数を数えます
num_one_targets = int(np.sum(targets_all))

# 数を数えるための変数を定義します
zero_targets_counter = 0

# バランスを取る上で不要なデータを入れるためのリストを定義します
indices_to_remove = []

# ターゲットの数を数え、その数を超える0のデータがあった場合はリストにその値をいれていきます
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# 入力とターゲットを入れるための変数を定義します
# また、バランスが取れていないデータを削除していきます
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### 入力の標準化

In [ ]:
# sklearnのモジュールを使って前処理をしていきます
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### データのシャッフル

In [ ]:
# データをバッチに分けていくことから、何らかの規則性が生まれないようにデータをシャッフルしていきます
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# シャッフルしたデータを変数に入れていきます
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### データセットの分割

In [ ]:
# データの数を数えます
samples_count = shuffled_inputs.shape[0]

# 80-10-10に分けることを前提として、訓練用と検証用のデータの数を定義していきます
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# テスト用のデータの数を定義します
test_samples_count = samples_count - train_samples_count - validation_samples_count

# 訓練用データを作成します
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# 検証用データを作成します
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# テスト用データを作成します
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


# 作成したデータの中の１と０の割合を確認します
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

### データを*.npz形式で保存します

In [ ]:
# Ssavezメソッドを使ってデータを保存していきます
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)